# TransformerOverBERT

In [1]:
import joblib
import os
import optuna
import sys
import torch
import wandb

sys.path.insert(0, os.path.abspath('../'))

from rhetorical_roles_classification import (
    AbsoluteSinusoidalEmbedder,
    AutoTransformerOverBERTForTokenClassification,
    RelativeSinusoidalEmbedder,
    RhetoricalRolesDatasetForTransformerOverBERT
)
from rhetorical_roles_classification.train import train
from rhetorical_roles_classification.test import test_ToBERT

from config import (
    DATA_FOLDER,
    MODELS_FOLDER,
    HUGGING_FACE_BERT_MODEL,
    NUM_LABELS,
    WANDB_PROJECT,
    MAX_SEGMENT_LENGTH,
    label2rhetRole
)

In [2]:
MODEL_NAME = "ToBERT"

MAX_DOCUMENT_LENGTH = 386

DEVICE = "cuda:1"

wandb_login_success = wandb.login()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: gmarinowandb-8601. Use `wandb login --relogin` to force relogin


## Dataset Loading

In [3]:
train_dataset = RhetoricalRolesDatasetForTransformerOverBERT(
    data_filepath=os.path.join(DATA_FOLDER, "train.json"),
    max_document_length=MAX_DOCUMENT_LENGTH,
    max_segment_length=MAX_SEGMENT_LENGTH,
    tokenizer_model_name=HUGGING_FACE_BERT_MODEL
)
valid_dataset = RhetoricalRolesDatasetForTransformerOverBERT(
    data_filepath=os.path.join(DATA_FOLDER, "valid.json"),
    max_document_length=MAX_DOCUMENT_LENGTH,
    max_segment_length=MAX_SEGMENT_LENGTH,
    tokenizer_model_name=HUGGING_FACE_BERT_MODEL
)
test_dataset = RhetoricalRolesDatasetForTransformerOverBERT(
    data_filepath=os.path.join(DATA_FOLDER, "test.json"),
    max_document_length=MAX_DOCUMENT_LENGTH,
    max_segment_length=MAX_SEGMENT_LENGTH,
    tokenizer_model_name=HUGGING_FACE_BERT_MODEL
)

## Embedders

In [ ]:
absolute_embedder = AbsoluteSinusoidalEmbedder(
    max_document_length=MAX_DOCUMENT_LENGTH,
    embedding_dimension=768
)
joblib.dump(absolute_embedder, os.path.join(MODELS_FOLDER, "absolute_embedder"))

relative_embedder = RelativeSinusoidalEmbedder(
    max_document_length=MAX_DOCUMENT_LENGTH,
    embedding_dimension=768
)
joblib.dump(relative_embedder, os.path.join(MODELS_FOLDER, "relative_embedder"))

In [4]:
absolute_embedder = joblib.load(os.path.join(MODELS_FOLDER, "absolute_embedder"))
relative_embedder = joblib.load(os.path.join(MODELS_FOLDER, "relative_embedder"))

## Hyperparameter Tuning

In [5]:
hyperparameters = {
    "batch_size": 1,
    "epochs": 20,
    "num_warmup_steps": 0,
    "accum_iter": 3,
    "early_stopping": True,
    "early_stopping_patience": 2
}

def objective(trial):
    
    print("\n************************ Starting New Optuna Trial ************************\n")
    
    hyperparameters["lr"] = trial.suggest_float("lr", 5e-6, 5e-4)
    hyperparameters["weight_decay"] = trial.suggest_float("weight_decay", 1e-3, 1e-1)
    
    embedding_type = trial.suggest_categorical("embedding", ["abs", "rel"])
    embedder = absolute_embedder if embedding_type == "abs" else relative_embedder

    transformer_dropout = trial.suggest_float("transformer_dropout", 0.1, 0.7)
    transformer_dim_feedforward = trial.suggest_int("transformer_dim_feedforward", 50, 1000)
    transformer = torch.nn.Transformer(
        d_model=768,
        nhead=12,
        batch_first=True,
        dim_feedforward=transformer_dim_feedforward,
        activation="gelu",
        dropout=transformer_dropout,
        layer_norm_eps=1e-12,
        num_encoder_layers=2,
        num_decoder_layers=0
    ).encoder

    model = AutoTransformerOverBERTForTokenClassification(
        model_name=HUGGING_FACE_BERT_MODEL,
        embedder=embedder,
        transformer=transformer,
        num_labels=NUM_LABELS,
        max_document_length=MAX_DOCUMENT_LENGTH,
        max_segment_length=MAX_SEGMENT_LENGTH,
        device=DEVICE
    )
    model_id = f"{MODEL_NAME}-Trial{trial.number}"

    with wandb.init(project=WANDB_PROJECT, config=hyperparameters, name=model_id):

        wandb.config["embedding_type"] = embedding_type
        wandb.config["transformer_dropout"] = transformer_dropout
        wandb.config["transformer_dim_feedforward"] = transformer_dim_feedforward        

        valid_loss = train(
            model=model,
            destination_path=os.path.join(MODELS_FOLDER, model_id),
            train_dataset=train_dataset,
            valid_dataset=valid_dataset,
            device=DEVICE,
            optuna_trial=trial,
            use_wandb=True,
            **hyperparameters
        )
        
    return valid_loss

In [5]:
study_name = f"{MODEL_NAME}.optuna"
storage_name = "sqlite:///{}.db".format(study_name)
study = optuna.create_study(study_name=study_name, storage=storage_name, load_if_exists=True, direction="minimize")

[I 2023-04-29 12:26:58,535] Using an existing study with name 'ToBERT.optuna' instead of creating a new one.


In [7]:
study.optimize(objective, n_trials=32)


************************ Starting New Optuna Trial ************************



Some weights of the model checkpoint at nlpaueb/legal-bert-base-uncased were not used when initializing BertForTokenClassification: ['cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initia


************** Training Started **************

EPOCH N. 1


221it [01:30,  2.44it/s]



TRAIN RESULTS
Loss: 1.9679547203072596
Accuracy: 0.42023920472196336
MCC: 0.19230760693160562
Macro F1: 0.11017042053172824
Macro Precision: 0.14173441118908806
Macro Recall: 0.12281069204929076
Micro F1: 0.42023920472196336
Micro Precision: 0.42023920472196336
Micro Recall: 0.42023920472196336

VALIDATION RESULTS
Loss: 2.1501386165618896
Accuracy: 0.5553494124922697
MCC: 0.3987340430629391
Macro F1: 0.14705571811251172
Macro Precision: 0.1400871570320531
Macro Recall: 0.16823164117976763
Micro F1: 0.5553494124922697
Micro Precision: 0.5553494124922697
Micro Recall: 0.5553494124922697
--------------------------------------------------
EPOCH N. 2


...


TRAIN RESULTS
Loss: 0.3933877304398636
Accuracy: 0.8470798384591488
MCC: 0.8054293188478819
Macro F1: 0.722419668179758
Macro Precision: 0.7632343578069618
Macro Recall: 0.6970061428538368
Micro F1: 0.8470798384591488
Micro Precision: 0.8470798384591488
Micro Recall: 0.8470798384591488

VALIDATION RESULTS
Loss: 0.8338495492935181
Accuracy: 0.7702535559678417
MCC: 0.7028481821197874
Macro F1: 0.5792665735341221
Macro Precision: 0.5951262028078088
Macro Recall: 0.5750123431376014
Micro F1: 0.7702535559678417
Micro Precision: 0.7702535559678417
Micro Recall: 0.7702535559678417
--------------------------------------------------
EPOCH N. 7


221it [01:26,  2.54it/s]



TRAIN RESULTS
Loss: 0.2986084440158368
Accuracy: 0.8814461012736875
MCC: 0.8497335763168035
Macro F1: 0.7796792512975593
Macro Precision: 0.8381272218864321
Macro Recall: 0.7583833385927955
Micro F1: 0.8814461012736876
Micro Precision: 0.8814461012736875
Micro Recall: 0.8814461012736875

VALIDATION RESULTS
Loss: 0.9299208521842957
Accuracy: 0.7687074829931972
MCC: 0.6988958496760639
Macro F1: 0.5589906250025243
Macro Precision: 0.5897640669594367
Macro Recall: 0.5387373049561919
Micro F1: 0.7687074829931972
Micro Precision: 0.7687074829931972
Micro Recall: 0.7687074829931972
Early stopped training
--------------------------------------------------


Epoch,▁▂▃▅▆▇█
Train Loss,█▅▄▃▂▂▁
Validation Loss,█▃▂▂▁▂▄
Epoch,7
Train Loss,0.29861
Validation Loss,0.92992


[I 2023-04-25 20:05:30,269] Trial 31 finished with value: 0.7980014085769653 and parameters: {'lr': 8.89671135302201e-05, 'weight_decay': 0.08271398460897429, 'embedding': 'rel', 'transformer_dropout': 0.1482254169989244, 'transformer_dim_feedforward': 952}. Best is trial 14 with value: 0.7874695062637329.


In [6]:
print("\n************************ Optuna Study Completed ************************\n")
best_trial = study.best_trial
print("Best trial:", best_trial.number)
print("Value:", best_trial.value)
print("Params:")
for key, value in best_trial.params.items():
    print(f"\t{key}: {value}")


************************ Optuna Study Completed ************************

Best trial: 14
Value: 0.7874695062637329
Params:
	embedding: rel
	lr: 7.544257211555746e-05
	transformer_dim_feedforward: 968
	transformer_dropout: 0.13346778353454997
	weight_decay: 0.08363287013131492


## Test

In [7]:
best_trial = study.best_trial
best_params = best_trial.params

embedder = absolute_embedder if best_params["embedding"] == "abs" else relative_embedder

transformer = torch.nn.Transformer(
    d_model=768,
    nhead=12,
    batch_first=True,
    dim_feedforward=best_params["transformer_dim_feedforward"],
    activation="gelu",
    dropout=best_params["transformer_dropout"],
    layer_norm_eps=1e-12,
    num_encoder_layers=2,
    num_decoder_layers=0
).encoder

model = AutoTransformerOverBERTForTokenClassification(
    model_name=HUGGING_FACE_BERT_MODEL,
    embedder=embedder,
    transformer=transformer,
    num_labels=NUM_LABELS,
    max_document_length=MAX_DOCUMENT_LENGTH,
    max_segment_length=MAX_SEGMENT_LENGTH,
    device=DEVICE
)

model_id = f"{MODEL_NAME}-Trial{best_trial.number}"
model.load_state_dict(torch.load(os.path.join(MODELS_FOLDER, model_id)))

Some weights of the model checkpoint at nlpaueb/legal-bert-base-uncased were not used when initializing BertForTokenClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initia

<All keys matched successfully>

In [8]:
test_ToBERT(
    model=model,
    test_dataset=test_dataset,
    label2rhetRole=label2rhetRole,
    device=DEVICE,
)

100%|██████████| 30/30 [00:11<00:00,  2.58it/s]

Overall
	Accuracy: 0.7846474470302188
	MCC: 0.7267670113883938
	Macro F1: 0.5738055350853982
	Macro Precision: 0.6233189100752514
	Macro Recall: 0.564134905480945
	Micro F1: 0.7846474470302188
	Micro Precision: 0.7846474470302188
	Micro Recall: 0.7846474470302188
Rhetorical role: PREAMBLE
	Macro F1: 0.972360717424781
	Macro Precision: 0.9650979366930781
	Macro Recall: 0.9800347539328567
Rhetorical role: FAC
	Macro F1: 0.8728314702807327
	Macro Precision: 0.8770990744745795
	Macro Recall: 0.868752906061106
Rhetorical role: RLC
	Macro F1: 0.7119394276215356
	Macro Precision: 0.7941883986660105
	Macro Recall: 0.6678897250614649
Rhetorical role: ISSUE
	Macro F1: 0.886315915977784
	Macro Precision: 0.875589205357262
	Macro Recall: 0.8977023683280312
Rhetorical role: ARG_PETITIONER
	Macro F1: 0.5753200137675577
	Macro Precision: 0.6372785829307568
	Macro Recall: 0.553760870670803
Rhetorical role: ARG_RESPONDENT
	Macro F1: 0.6979748896386189
	Macro Precision: 0.6810762903052533
	Macro Recall: